# MAHNOB functions exploration

Taking and look at several methods made for the gaze data. The methods are in `find_fixations_saccades.py`

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from find_fixations_saccades import *

sns.set(rc={'figure.figsize':(11.7,8.27)})

The code in this notebook was tested against session 10 data, but should work for all gaze data.

In [ ]:
# Change accordingly
session_path = "10/P1-Rec1-All-Data-New_Section_10.tsv"

In [ ]:
dataset_path = "/net2/expData/affective_eeg/mahnob_dataset/Sessions"
data_path = os.path.join(dataset_path, session_path)

gaze = pd.read_csv(data_path, sep="\t", header=23)
gaze = gaze.drop(columns=["Unnamed: 43", "Unnamed: 44"])

media_width = gaze.loc[0,"MediaWidth"]
media_height = gaze.loc[0,"MediaHeight"]

# Reverse Y due to coordinate system of raw data
gaze["MappedFixationPointYRev"] = gaze.MappedFixationPointY.apply(lambda x: media_height - x)
gaze["MappedGazeDataPointYRev"] = gaze.MappedGazeDataPointY.apply(lambda y: media_height - y)

In [ ]:
gaze.columns

## Blink detection

First we look at the validity score provided by Tobii for each frame.

In [ ]:
print(gaze.loc[(gaze.ValidityRight == 4) & (gaze.ValidityLeft == 4), "MappedGazeDataPointX"].unique())
print(gaze.loc[(gaze.ValidityRight == 4) & (gaze.ValidityLeft == 4), "MappedGazeDataPointY"].unique())

So whenever the validity of gaze point is bad, the gaze coordinate is set to (0, 0).

In [ ]:
gaze.loc[(gaze.ValidityRight == 4) & (gaze.ValidityLeft == 4), ["Timestamp", "MappedGazeDataPointX", "MappedGazeDataPointY"]]

Now we try out the blink detection function.

In [ ]:
blink_from_gaze = blink_detection(gaze.MappedGazeDataPointX, gaze.MappedGazeDataPointY, gaze.Timestamp)[1]
blink_from_validity = blink_detection(gaze.ValidityLeft, gaze.ValidityRight, gaze.Timestamp, missing=4)[1]

In [ ]:
blink_from_gaze == blink_from_validity

So it seems that the validity score itself can be used to extract blinks.

Look at the number of blinks extracted by this method.

In [ ]:
print("Number of potential blinks detected:", len(blink_from_gaze))

That seems to be a small number. Average eye blink in relaxation is about 12-15 per minute. The recording is about 3 minutes.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6118863

May need to find a way to better tell the duration of typical eye blink.

The number semes to be better by reducing the minimal length of blink.

In [ ]:
blink_from_validity = blink_detection(gaze.ValidityLeft, gaze.ValidityRight, gaze.Timestamp, missing=4, minlen=1)[1]

print("Number of potential blinks detected:", len(blink_from_validity))

## Find fixation

Now try the fixation algorithm.

In [ ]:
fixation_from_gaze = np.array(fixation_detection(gaze.MappedGazeDataPointX, gaze.MappedGazeDataPointYRev, gaze.Timestamp)[1])

print("Fixations found from functions provided:", len(fixation_from_gaze))

That's more than what Tobii found.

In [ ]:
sns.scatterplot(x=fixation_from_gaze[:,3], y=fixation_from_gaze[:,4],
                hue=fixation_from_gaze[:,2], size=fixation_from_gaze[:,2])
plt.ylim(0, media_height)
plt.xlim(0, media_width)
plt.show()

Similar to the Tobii fixation result, but with more points.

## Find saccade

In [ ]:
saccade_from_gaze = pd.DataFrame(data=saccade_detection(gaze.MappedGazeDataPointX, gaze.MappedGazeDataPointY, gaze.Timestamp)[1],
                                 columns=["starttime", "endtime", "duration", "startx", "starty", "endx", "endy"])
saccade_from_gaze.head()

In [ ]:
print('Saccades found:', len(saccade_from_gaze))